# VacationPy
---

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("WeatherPy_Output/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazovsky,67.4667,78.7000,-21.37,90,80,4.37,RU,1706555452
1,1,port-aux-francais,-49.3500,70.2167,3.71,72,99,16.49,TF,1706555453
2,2,shar,49.5886,81.0453,-18.09,99,40,1.40,KZ,1706555453
3,3,edinburgh of the seven seas,-37.0676,-12.3116,16.84,75,40,9.91,SH,1706555454
4,4,polyarnyy,69.1989,33.4478,1.28,67,100,15.44,RU,1706555454


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.9)


# Display the map
map_plot_1 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_weather_df))

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

11


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,la ronge,55.1001,-105.2842,4.14,56,11,1.54,CA,1706555469
203,203,vadso,70.0744,29.7487,2.49,69,30,19.55,NO,1706555581
271,271,al jawf,29.5000,38.7500,11.51,46,2,2.58,SA,1706555622
284,284,hammerfest,70.6634,23.6821,2.25,60,40,25.21,NO,1706555281
323,323,kristiansund,63.1115,7.7320,6.36,56,0,6.69,NO,1706555608


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
27,la ronge,CA,55.1001,-105.2842,56,
203,vadso,NO,70.0744,29.7487,69,
271,al jawf,SA,29.5000,38.7500,46,
284,hammerfest,NO,70.6634,23.6821,60,
323,kristiansund,NO,63.1115,7.7320,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
la ronge - nearest hotel: No hotel found
vadso - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
hammerfest - nearest hotel: No hotel found
kristiansund - nearest hotel: No hotel found
stjordalshalsen - nearest hotel: No hotel found
gumdag - nearest hotel: No hotel found
florence - nearest hotel: No hotel found
castel bolognese - nearest hotel: No hotel found
alesund - nearest hotel: No hotel found
swift current - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
27,la ronge,CA,55.1001,-105.2842,56,No hotel found
203,vadso,NO,70.0744,29.7487,69,No hotel found
271,al jawf,SA,29.5000,38.7500,46,No hotel found
284,hammerfest,NO,70.6634,23.6821,60,No hotel found
323,kristiansund,NO,63.1115,7.7320,56,No hotel found
331,stjordalshalsen,NO,63.4681,10.9262,56,No hotel found
348,gumdag,TM,39.2061,54.5906,68,No hotel found
366,florence,IT,43.7667,11.2500,56,No hotel found
388,castel bolognese,IT,44.3207,11.7999,67,No hotel found
393,alesund,NO,62.4723,6.1549,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            s='Humidity',
            scale= 1.2)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)